#  SETUP

In [ ]:
# if the script finishes very quickly (and generates an empty excel file), click run again
# if the script errors on the "Login Cell" (added a comment to indicate which cell that is below), set IS_HEADLESS to "False" and run again. The scraper will automatically launch a page and attempt to login to LinkedIn. It's likely erroring because LinkedIn is asking for a captcha to verify the user is not a bot. Solve the captch/challenge and login. Once successfully logged in, set IS_HEADLESS back to "True" and run again.

import pickle
import csv
import os
import random
import time
from linkedin_scraper import actions
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException
from dotenv import load_dotenv
import pandas as pd
# for google docs upload
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from io import StringIO


IS_HEADLESS = False

# Set environment variables to email and password
load_dotenv()
LINKEDIN_EMAIL = os.environ.get('LINKEDIN_EMAIL')
LINKEDIN_PASSWORD = os.environ.get('LINKEDIN_PASSWORD')
# Check if the environment variables are set
if not LINKEDIN_EMAIL or not LINKEDIN_PASSWORD:
    raise ValueError("LinkedIn credentials not set in environment variables")


COMPANY_CATEGORIES = {
    "SECURITY": [
        "Okta",
        "Snyk",
        "R2C/Semgrep", 
        "Wiz",
        "Lacework",
        "Crowdstrike",
        "Palo Alto Networks",
        "Island",
        "Vanta",
        "Material Security",
        "Abnormal Security",
        "Samsara",
    ],
    "OTHER": [
        "Figma",
        "Airtable",
        "Notion",
        "Canva",
        "Webflow",
        "Faire",
        "Deel",
        "Rippling",
        "Flexport",
        "Benchling",
        "Solugen"
    ],
    "PUBLIC": [
        "Doordash",
        "Uber",
        "Palantir",
        "Airbnb",
        "Instacart"
    ],
    "INFRA": [
        "Fivetran",
        "DBT",
        "Temporal",
        "Cockroach Labs",
        "Grafana",
        "Zapier",
        "Starburst",
        "Retool",
        "Sentry",
        "Sourcegraph",
        "Cribl",
        "Vercel",
        "Clickhouse",
        "Github,"
        "Cisco Meraki",
    ],
    "FINTECH": [
        "Robinhood",
        "Square",
        "Stripe",
        "Ramp",
        "Brex",
        "Plaid",
        "Modern Treasury",
        "Mercury",
        "Persona",
        "Klarna",
        "Nubank"
    ],
    "CRYPTO": [
        "Coinbase",
        "Uniswap",
        "Chainalysis",
        "Arbitrum",
        "TRM",
        "Fireblocks",
        "Eigenlayer"
    ],
    "FRONTIER": [
        "Anduril",
        "SpaceX",
        "Zipline",
        "Varda",
        "Hadrian"
    ],
    "AI": [
        "Bytedance",
        "Scale AI",
        "Anthropic",
        "Robust intelligence",
        "OpenAI",
        "Predibase",
        "Cohere",
        "Databricks",
        "Hugging Face",
        "RunwayML",
        "Tecton",
        "Weights & Biases",
        "Kumo AI",
        "NVIDIA",
        "Adept",
        "Glean",
        "Character.ai",
        "Midjourney",
        "Facebook AI",
        "FAIR",
        "Google brain"
    ]
}

In [ ]:
import sys
print(sys.executable)

# LOGIN

In [ ]:
# IF THIS CELL ERRORS DUE TO CAPTCHA, do this:
# manually complete the captcha, click on the next cell, and select Run menu, select "run selected cell and all below" 

def instantiate_driver():
    options = FirefoxOptions()
    if IS_HEADLESS:
        options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)
    
    try:
        actions.login(driver, LINKEDIN_EMAIL, LINKEDIN_PASSWORD)
    except Exception as e:
        print("Error logging in. Please complete the captcha challenge and login manually.")
        print(e)

    time.sleep(15)
    return driver
    
driver = instantiate_driver()

# SALES NAV HELPER FUNCTIONS

In [ ]:
class ScrapedProfile:
    def __init__(self, profile_name, experiences, education, profile_dist, mutuals, profile_description, profile_link):
        self.profile_name = profile_name
        self.experiences = experiences
        self.education = education
        self.profile_dist = profile_dist
        self.mutuals = mutuals
        self.profile_description = profile_description
        self.profile_link = profile_link

def check_pickles(historyPickle, toScrapePickle):
    scraped_urls = []
    with open(historyPickle, 'rb') as f:
        try:
            scraped_urls = pickle.load(f)
        except Exception as e:
            print(e)
            print("Error loading history pickle file")
        print(len(scraped_urls))

    to_scrape_urls = []
    with open(toScrapePickle, 'rb') as f:
        try:
            to_scrape_urls = pickle.load(f)
        except Exception as e:
            print(e)
            print("Error loading to scrape url pickle file")
        print(len(to_scrape_urls))

    return scraped_urls, to_scrape_urls

def deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls):
    to_scrape_urls = list(set(to_scrape_urls))
    originalCount = len(to_scrape_urls)
    to_scrape_urls = [url for url in to_scrape_urls if url not in already_scraped_urls]
    print(f"Removed {originalCount - len(to_scrape_urls)} duplicates")

    with open(toScrapePickle, 'wb') as to_scrape_file:
        pickle.dump(to_scrape_urls, to_scrape_file)
    
    print(len(to_scrape_urls))


"""
Wait for an element to be present on the page and return it.

Parameters:
- driver: The WebDriver instance
- by: The method to locate the element (default: By.CLASS_NAME)
- name: The name or identifier of the element to wait for
- base: The base element to search from (default: None, which uses the driver)
- timeout: Maximum time to wait for the element (default: 180 seconds)

Returns:
- The WebElement if found
- None if the element is not found within the timeout period
"""
def wait_for_element_to_load(driver, by=By.CLASS_NAME, name="pv-top-card", base=None, timeout=100):
    base = base or driver
    try:
        element = WebDriverWait(base, timeout).until(
            EC.presence_of_element_located((by, name))
        )
        return element
    except TimeoutException:
        print(f"Timed out waiting for element: {by}={name}")
        return None
    except Exception as e:
        print(f"An error occurred while waiting for element {by}={name}: {str(e)}")
        return None

# Scrolls the page to bring the specified profile element into view
# Returns True if successful, False if an error occurs
def scroll_to_profile(driver, profile):
    try:
        driver.execute_script("arguments[0].scrollIntoView();", profile)
        return True
    except Exception as e:
        print(f"Error scrolling to profile: {e}")
        return False

# Clicks on the profile element to open its details
# Returns True if successful, False if an error occurs
def click_profile(profile):
    try:
        salesNavOpenProfileButton = profile.find_element(By.CLASS_NAME, "artdeco-entity-lockup__title")
        salesNavOpenProfileButton.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Error clicking profile: {e}")
        return False

# Clicks the three dots button to open the dropdown menu
# Returns True if successful, False if an error occurs
def click_three_dots_button(driver):
    try:
        actionContainer = driver.find_element(By.CLASS_NAME, "_actions-container_1dg5u8")
        threeDotsButton = actionContainer.find_element(By.CLASS_NAME, "_icon_ps32ck")
        threeDotsButton.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Error clicking three dots button")
        return False

# Retrieves the LinkedIn URL from the dropdown menu
# Returns the URL if successful, None if an error occurs
def get_linkedin_url(driver):
    try:
        dropdownContainer = driver.find_element(By.CLASS_NAME, "_visible_x5gf48")
        normalLinkedInUrl = dropdownContainer.find_elements(By.TAG_NAME, "a")[1].get_attribute("href")
        return normalLinkedInUrl
    except (NoSuchElementException, IndexError) as e:
        print(f"Error getting LinkedIn URL: {e}")
        return None

# Closes the profile popout
# Returns True if successful, False if an error occurs
def close_popout(driver):
    try:
        header = driver.find_element(By.CLASS_NAME, "_inline-sidesheet-header-actions_1cn7lg")
        button = header.find_elements(By.CLASS_NAME, "_button_ps32ck")[1]
        button.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException, IndexError) as e:
        print(f"Error closing popout: {e}")
        return False

# Navigates to the next page of search results
# Returns True if successful, False if there are no more pages or an error occurs
def navigate_to_next_page(driver):
    try:
        nextPageButton = driver.find_element(By.CLASS_NAME, "artdeco-pagination__button--next")
        nextPageButton.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"No more pages or error navigating: {e}")
        return False

# Main scraping function
def scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle):
    # reinstantiate_driver(driver)
    driver.get(SALES_NAV_SEARCH_URL)
    
    while True:
        wait_for_element_to_load(driver, By.ID, "search-results-container")
        profiles = driver.find_elements(By.CLASS_NAME, "artdeco-list__item")
        
        for profile in profiles:
            if not scroll_to_profile(driver, profile):
                continue

            # time.sleep(2 + random.random() * 6)


            if not wait_for_element_to_load(driver, By.CLASS_NAME, "artdeco-entity-lockup__title"):
                continue

            if not click_profile(profile):
                continue
            
            # time.sleep(2 + random.random() * 6)

            if not wait_for_element_to_load(driver, By.CLASS_NAME, "_actions-container_1dg5u8"):
                continue

            if not click_three_dots_button(driver):
                continue

            # time.sleep(2 + random.random() * 6)


            if not wait_for_element_to_load(driver, By.CLASS_NAME, "_visible_x5gf48"):
                continue

            normalLinkedInUrl = get_linkedin_url(driver)
            if normalLinkedInUrl:
                if normalLinkedInUrl in already_scraped_urls:
                    print("Skipping (already scraped): " + normalLinkedInUrl)
                else:
                    to_scrape_urls.append(normalLinkedInUrl)
                    with open(toScrapePickle, 'wb') as f:
                        pickle.dump(to_scrape_urls, f)
                    print("Successfully scraped: " + normalLinkedInUrl)

            if not close_popout(driver):
                continue

            # time.sleep(2 + random.random() * 6)


        next_button = wait_for_element_to_load(driver, By.CLASS_NAME, "artdeco-pagination__button--next")
        if not next_button or not next_button.is_enabled():
            break

        next_button.click()

        if not wait_for_element_to_load(driver, By.CLASS_NAME, "artdeco-list__item"):
            break

    return to_scrape_urls

# PROFILE SCRAPING HELPER FUNCTIONS

In [ ]:
def filter():
    pass     
    # try:
        #     WebDriverWait(driver, 240).until(lambda d: d.execute_script('return document.readyState') == 'complete')
        # except:
        #     driver = reinstantiate_driver()

        # # FILTERING
        
        # likely_founder = True
        # relevant_exp = True

        # cur_exp = experiences[0]
        # relevant_companies = ["stealth", "new"]
        # if any(company in cur_exp.institution_name.split(" ·")[0].lower() for company in relevant_companies) or "present" not in cur_exp.to_date.lower():
        #     likely_founder = True

        # relevant_titles = ["product", "engineer", "sales", "business development", "founder", "head", "lead", "senior", "staff", "chief", "growth"]
        # for experience in experiences[1:5]:
        #     if any(title in experience.position_title.lower() for title in relevant_titles):
        #         relevant_exp = True
        #         break
        # relevant_exp = True

        # if not (likely_founder and relevant_exp):
        #     print(likely_founder, relevant_exp)
        #     return None

        # person_obj = Person(profile_link, driver = driver, scrape=False, experiences = [None])
        # try:
        #     WebDriverWait(driver, 240).until(lambda d: d.execute_script('return document.readyState') == 'complete')
        # except:
        #     driver = reinstantiate_driver(driver)
        # time.sleep(2 + random.random() * 7)
        
# Helper function to scrape experiences
# Returns scraped experiences if successful, otherwise returns empty list
def get_experiences(driver):
    scraped_experiences = []

    try:
        wait_for_element_to_load(driver, By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")
        experience_items = driver.find_elements(By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")

        if len(experience_items) > 0:
            for item in experience_items:
                hidden_spans = item.find_elements(By.CSS_SELECTOR, "span.visually-hidden")
                experience_texts = [span.text for span in hidden_spans]
                experience = {
                    "title: ": experience_texts[0],
                    "company: ": experience_texts[1],
                    "dates: ": experience_texts[2],
                }
                
                if len(experience_texts) > 3:
                    experience["location: "] = experience_texts[3]
                if len(experience_texts) > 4:
                    experience["summary: "] = experience_texts[4]
                if len(experience_texts) > 5:
                    experience["remaining: "] = (", ").join(experience_texts[5:])
                scraped_experiences.append(experience)

            print("Successfully scraped experiences")
        else:
            print("No experiences found")

    except Exception as e:
        print(e)
        print("ERROR: No experiences found")

    return scraped_experiences


# Helper function to scrape education
# Returns scraped education if successful, otherwise returns empty list
def get_education(driver):
    scraped_education = []

    try:
        wait_for_element_to_load(driver, By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")
        education_items = driver.find_elements(By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")

        if len(education_items) > 0:
            for item in education_items:
                hidden_spans = item.find_elements(By.CSS_SELECTOR, "span.visually-hidden")
                education_texts = [span.text for span in hidden_spans]
                education = {
                    "school: ": education_texts[0],
                }
                
                if len(education_texts) > 1:
                    education["degree: "] = education_texts[1]
                if len(education_texts) > 2:
                    education["dates: "] = education_texts[2]
                if len(education_texts) > 3:
                    education["remaining: "] = (", ").join(education_texts[5:])
                scraped_education.append(education)

            print("Successfully scraped education")
        else:
            print("No education found")

    except Exception as e:
        print(e)
        print("ERROR: No education found")

    return scraped_education


# Helper function to scrape degree of connection and mutuals
# Returns scraped degree of connection and mutuals if successful, otherwise returns N/A
def get_degree_of_connection_and_mutuals(driver):
    scraped_profile_dist = "4+"
    scraped_mutuals = "N/A"

    try:
        scraped_profile_dist = wait_for_element_to_load(driver, By.CSS_SELECTOR, "span.dist-value").text
        if scraped_profile_dist == "1st" or scraped_profile_dist == "2nd":
            try:
                span_element = wait_for_element_to_load(driver, By.CSS_SELECTOR, "span.t-normal.t-black--light.t-14.hoverable-link-text")
                scraped_mutuals = span_element.text.split('\n')[0]
                print("Successfully found mutual connections: " + scraped_mutuals)
            except:
                print("ERROR: mutuals not found")
        print("Successfully scraped degree of connection: " + scraped_profile_dist)

    except Exception as e:
        print(e)
        print("No degree of connection found")

    return scraped_profile_dist, scraped_mutuals

# Helper function to scrape description
# Returns scraped description if successful, otherwise returns N/A
def get_description(driver):
    scraped_description = "N/A"
    
    try:
        scraped_description = driver.find_element(By.CLASS_NAME, "text-body-medium.break-words").text
        print("Successfully scraped description: " + scraped_description)
    except Exception as e:
        print(e)
        print("ERROR: description not found")
        
    return scraped_description

# Main function to scrape degree of connection and mutuals
# Returns scraped degree of connection and mutuals if successful, otherwise returns N/A
def scrape_profile(driver, scraped_link):

    # Scrape Name
    driver.get(scraped_link)
    scraped_profile_name = wait_for_element_to_load(driver, By.CSS_SELECTOR, "h1.text-heading-xlarge.inline.t-24.v-align-middle.break-words").text
    time.sleep(2 + random.random() * 6)

    # Scrape Experiences
    experiences_url = os.path.join(scraped_link, "details/experience")
    driver.get(experiences_url)
    scraped_experiences = get_experiences(driver)
    time.sleep(2 + random.random() * 6)


    # Scrape Education
    education_url = os.path.join(scraped_link, "details/education")
    driver.get(education_url)
    scraped_education = get_education(driver)
    time.sleep(2 + random.random() * 6)


    # Scrape degree of connection and mutuals if available
    driver.get(scraped_link)
    scraped_profile_dist, scraped_mutuals = get_degree_of_connection_and_mutuals(driver)
    time.sleep(2 + random.random() * 6)

    # Scrape description
    scraped_description = get_description(driver)
    time.sleep(2 + random.random() * 6)


    # Scape profile link
    scraped_link = driver.current_url
    print("Successfully scraped profile link: " + scraped_link)
    time.sleep(2 + random.random() * 6)

    
    profile = ScrapedProfile(scraped_profile_name,
                   scraped_experiences,
                   scraped_education,
                   scraped_profile_dist,
                   scraped_mutuals,
                   scraped_description,
                   scraped_link)
    print("\nSuccess!")
    print(f"Name: {profile.profile_name}")
    print(f"Experiences: {profile.experiences}")
    print(f"Education: {profile.education}")
    print(f"Profile Distance: {profile.profile_dist}")
    print(f"Mutuals: {profile.mutuals}")
    print(f"Description: {profile.profile_description}")
    print(f"Link: {profile.profile_link}\n")
    return profile

def scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile):
    results = []
    scraped_urls = []
    history = []

    with open(toScrapePickle, 'rb') as f:
        try:
            to_scrape_urls = pickle.load(f)
        except Exception as e:
            print(e)
            print("Error loading to scrape url pickle file")
        
    totalCount = len(to_scrape_urls)

    print(f'# of profiles to scrape: {totalCount}')
    

    for i in range(len(to_scrape_urls) - 1, -1, -1):
        url = to_scrape_urls[i]
        print(f'At index: {len(to_scrape_urls) - i} - url: {url}')

        with open(historyPickle, 'rb') as f:
            try:
                history = pickle.load(f)
            except Exception as e:
                print(e)
                print("Error loading history pickle file")

        try:
            profile = scrape_profile(driver, url)
            if profile != None:
                print("saving profile info")
                results.append(profile)

                with open(resultsPickle, 'wb') as f:
                    pickle.dump(results, f)
                
                print("adding to history")
                history.append(url)
                with open(historyPickle, 'wb') as f:
                    pickle.dump(history, f)
                
                print("recording scraped url")
                scraped_urls.append(url)
            else:
                print("profile filtered out")
            
            print("removing from to-scrape")
            to_scrape_urls.remove(url)
            with open(toScrapePickle, 'wb') as f:
                pickle.dump(to_scrape_urls, f)

        except Exception as e:
            print(e)
            print('Failed to scrape profile: ', url)

            # TODO: CHANGED FAILED URLS
            with open(failedURLsTextFile, 'a') as f:
                f.write(url + '\n')

        print(((totalCount - i)/totalCount) * 100, '% Done - at index:', totalCount - i)
        print('\n----------------------------------------------------------------------------------------------------------\n')
    return results

# EXPORTING HELPER FUNCTIONS

In [ ]:
def parseProfile(profile: ScrapedProfile):
    res = {}
    res['url'] = profile.profile_link
    res['name'] = profile.profile_name
    res['dist'] = profile.profile_dist
    res['description'] = profile.profile_description

    #Experiences first
    print(profile.experiences)
    for i, e in enumerate(profile.experiences):
        res[f'exp{i} title'] = e["title: "]
        res[f'exp{i} company'] = e["company: "]
        res[f'exp{i} dates'] = e["dates: "]

    # School second
    for i, e in enumerate(profile.profile_school):
        res[f'edu{i} school'] = e["school: "]
        if "degree :" in e:
            res[f'edu{i} degree'] = e["degree: "]

    
    return res

# QUERY 1: MAIN COMPANIES

In [ ]:
historyPickle = 'db/main_history.pickle'
toScrapePickle = 'db/main_to_scrape.pickle'
resultsPickle = 'db/main_results.pickle'
failedURLsTextFile = 'db/main_failed_urls.txt'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people?savedSearchId=1833971580&sessionId=jFoOWCqiTl2JVJ3QPZrL4g%3D%3D
"""

### Sales nav scraper

In [ ]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

In [ ]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Deduplicate

In [ ]:
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

### Profile scraper

In [ ]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)
print(len(results))

# QUERY 2: UNICORN COMPANIES

In [ ]:
historyPickle = 'db/unicorn_history.pickle'
toScrapePickle = 'db/unicorn_to_scrape.pickle'
resultsPickle = 'db/unicorn_results.pickle'
failedURLsTextFile = 'db/unicorn_failed_urls.txt'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people?savedSearchId=1833971588&sessionId=zVKTgf2GQk2owfFVzL9aFQ%3D%3D
"""

### Sales nav scraper

In [ ]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

In [ ]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Deduplicate

In [ ]:
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

### Profile scraper

In [ ]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)
print(len(results))

# QUERY 3: ACQUIRED COMPANIES

In [ ]:
historyPickle = 'db/acquired_history.pickle'
toScrapePickle = 'db/acquired_to_scrape.pickle'
resultsPickle = 'db/acquired_results.pickle'
failedURLsTextFile = 'db/acquired_failed_urls.txt'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people?savedSearchId=1833971612&sessionId=R7vv8AiYRXiqMseY1m3CwQ%3D%3D
"""

### Sales nav scraper

In [ ]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

In [ ]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Deduplicate

In [ ]:
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

### Profile scraper

In [ ]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)
print(len(results))

# QUERY 4: VC PORTFOLIO COMPANIES

In [ ]:
historyPickle = 'db/portfolio_history.pickle'
toScrapePickle = 'db/portfolio_to_scrape.pickle'
resultsPickle = 'db/portfolio_results.pickle'
failedURLsTextFile = 'db/portfolio_failed_urls.txt'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people?savedSearchId=1833971604&sessionId=zVKTgf2GQk2owfFVzL9aFQ%3D%3D
"""

### Sales nav scraper

In [ ]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

In [ ]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Deduplicate

In [ ]:
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

### Profile scraper

In [ ]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)
print(len(results))

# QUERY 5: INFRA COMPANIES

In [ ]:
historyPickle = 'db/infra_history.pickle'
toScrapePickle = 'db/infra_to_scrape.pickle'
resultsPickle = 'db/infra_results.pickle'
failedURLsTextFile = 'db/infra_failed_urls.txt'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people?savedSearchId=1833971620&sessionId=R7vv8AiYRXiqMseY1m3CwQ%3D%3D
"""

### Sales nav scraper

In [ ]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

In [ ]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Deduplicate

In [ ]:
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

### Profile scraper

In [ ]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)
print(len(results))

# EXPORTING RESULTS

In [ ]:
# NEW GOOGLE SHEETS INTEGRATION

def create_google_sheet(list_of_results, sheet_names):
    # Set up the credentials
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('path/to/your/credentials.json', scope)
    client = gspread.authorize(creds)

    # Create a new Google Sheet
    sheet = client.create('LinkedIn Scraped Profiles')
    
    for results, sheet_name in zip(list_of_results, sheet_names):
        # Create a new worksheet
        worksheet = sheet.add_worksheet(title=sheet_name, rows="1000", cols="20")

        # Parse candidates and prepare rows
        rows = []
        for candidate in results:
            row = parseCandidate(candidate)
            rows.append(row)

        if not rows:
            print(f"No data for sheet: {sheet_name}")
            continue

        # Get column titles
        col_titles = list(rows[0].keys())

        # Prepare the data for batch update
        cells = [col_titles]  # Start with the header row
        for row in rows:
            cells.append([row.get(col, '') for col in col_titles])

        # Update the sheet in batch
        cell_range = f'A1:{gspread.utils.rowcol_to_a1(len(cells), len(col_titles))}'
        worksheet.update(cell_range, cells)

        print(f"Successfully added data to sheet: {sheet_name}")

    # Delete the default "Sheet1" if it exists
    try:
        sheet.del_worksheet(sheet.worksheet("Sheet1"))
    except:
        pass

    print(f"Google Sheet created successfully. URL: {sheet.url}")
    return sheet.url

# Usage
list_of_results = [results1, results2, results3]  # Your lists of ScrapedProfile objects
sheet_names = ["Results1", "Results2", "Results3"]  # Names for each sheet
sheet_url = create_google_sheet(list_of_results, sheet_names)

In [ ]:
# Create dataframe
rows = []
for candidate in results:
    row = parseCandidate(candidate)
    rows.append(row)

# Write to CSV
col_titles = rows[0].keys()

try:
    with open('candidates.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, col_titles)
        dict_writer.writeheader()
        dict_writer.writerows(rows)
        print("Successfully exported to csv")
except Exception as e:
    print(f"Failed to export to csv. Error: {e}")

#Export to Excel
try:
    df.to_excel('candidates.xlsx', index=False)
    print("Exported to Excel")
except:
    print("Failed to export to Excel")

    
# update db/already_scraped.pickle
with open('db/already_scraped.pickle', 'rb') as f:
    already_scraped = pickle.load(f)
    print(f"Previously scraped: {len(already_scraped)}")
    already_scraped = already_scraped + already_scraped_urls
    already_scraped = list(set(already_scraped))
    print(f"Newly scraped: {len(already_scraped)}")
with open('db/already_scraped.pickle', 'wb') as f:
    pickle.dump(already_scraped, f)

In [ ]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.file']

def authenticate():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def upload_file_to_drive(file_path, file_name, mime_type):
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, mimetype=mime_type)

    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print('File ID: %s' % file.get('id'))

if __name__ == '__main__':
    file_path = 'path_to_your_local_excel_file.xlsx'  # Replace with the path to your local file
    file_name = 'your_excel_file.xlsx'  # Replace with the desired name for the file in Google Drive
    mime_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'

    upload_file_to_drive(file_path, file_name, mime_type)

In [ ]:
# categories = COMPANY_CATEGORIES

# # Example color mapping for categories
# category_colors = {
#     "SECURITY": 'red',
#     "OTHER": 'blue',
#     "PUBLIC": 'green',
#     "INFRA": 'yellow',
#     "FINTECH": 'orange',
#     "CRYPTO": 'purple',
#     "FRONTIER": 'cyan',
#     "AI": 'magenta'
# }

# # Create a reverse dictionary for easier lookup: {company: category}
# company_category = {}
# for category, companies in categories.items():
#     for company in companies:
#         company_category[company] = category

# # Modify the style function
# def highlight_by_category(val):
#     category = company_category.get(val)
#     if category:
#         color = category_colors.get(category, 'none')  # default to 'none' if no color is specified
#     else:
#         color = 'none'
#     return f'background-color: {color}'

# import re
# ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
# def find_illegal_characters(df):
#     for column in df.columns:
#         for idx, item in enumerate(df[column]):
#             if isinstance(item, str) and ILLEGAL_CHARACTERS_RE.search(item):
#                 # replace illegal characters with an empty string
#                 df[column][idx] = ILLEGAL_CHARACTERS_RE.sub('', item)
#     return df

# styled_df = df
# styled_df = find_illegal_characters(styled_df)
# styled_df = df.style.applymap(highlight_by_category)

# # Save the styled DataFrame to an Excel file
# # get today's date in MM-DD-YYYY format
# import datetime
# now = datetime.datetime.now()
# date = now.strftime("%m-%d-%Y")
# styled_df.to_excel(f'results/{date}_{start}-{end}.xlsx', engine='openpyxl', index=False)
